In [ ]:
import requests
import time
import json
from datetime import datetime
from tqdm import tqdm
from PIL import Image
import io
from bs4 import BeautifulSoup
import os
import base64
import google.generativeai as genai
from PIL import UnidentifiedImageError
import random
from dotenv import load_dotenv
load_dotenv()
def create_session_with_browser_cookies(discourse_url, cookies):
    """Creates a requests session with the provided cookies."""
    session = requests.Session()
    # Check if cookies are present before setting them
    if not all(cookies.values()):
        print("Error: One or more cookie environment variables are not set.")
        print("Please check your .env file.")
        exit()

    for name, value in cookies.items():
        session.cookies.set(name, value, domain=discourse_url.split('//')[1])
    return session

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def fetch_all_posts_from_topic(topic_id):
    """
    Fetch all posts from a Discourse topic using post_stream.stream post IDs.
    Returns the full list of post data.
    """
    try:
        # Step 1: Get initial topic JSON with metadata and some posts
        url = f"{DISCOURSE_URL}/t/{topic_id}.json"
        response = session.get(url, timeout=30)
        response.raise_for_status()
        topic_json = response.json()

        all_post_ids = topic_json.get("post_stream", {}).get("stream", [])
        fetched_posts = topic_json.get("post_stream", {}).get("posts", [])
        fetched_post_ids = {p['id'] for p in fetched_posts}

        remaining_ids = [pid for pid in all_post_ids if pid not in fetched_post_ids]
        all_posts = list(fetched_posts)

        # Step 2: Paginate through the remaining post IDs in chunks
        chunk_size = 20
        for i in range(0, len(remaining_ids), chunk_size):
            chunk = remaining_ids[i:i + chunk_size]
            chunk_url = f"{DISCOURSE_URL}/t/{topic_id}/posts.json"
            params = {'post_ids[]': chunk}
            chunk_resp = session.get(chunk_url, params=params, timeout=30)
            chunk_resp.raise_for_status()
            chunk_data = chunk_resp.json()
            all_posts.extend(chunk_data.get('post_stream', {}).get('posts', []))

            time.sleep(0.5)  # Respectful delay

        return all_posts

    except Exception as e:
        tqdm.write(f"Error fetching posts from topic {topic_id}: {e}")
        return []


def get_text_from_image_data(image_data, max_retries=10):
    """Uses Gemini API to extract embedded text from image data with retry on failure."""
    if os.getenv("ENABLE_OCR", "1") != "1":
        return ""

    attempt = 0
    while attempt < max_retries:
        try:
            model = genai.GenerativeModel("gemini-2.0-flash")

            # Validate and re-open image
            img = Image.open(io.BytesIO(image_data))
            img.verify()
            img = Image.open(io.BytesIO(image_data))
            if len(image_data) < 2000:

                print("[Skip] Image too small, likely emoji or icon")
                return ""
            # Convert to PNG bytes
                if (
                  'avatar' in img_url or
                  img_url.endswith('.svg') or
                  'emoji' in img_url or
                  'emoji' in img_tag.get('class', []) or
                  '/images/emoji' in img_url):
                  tqdm.write(f"  [Skipped] Emoji/system image: {img_url}")
                  continue

            buffered = io.BytesIO()
            img.save(buffered, format="PNG")
            image_bytes = buffered.getvalue()

            # Call Gemini API
            response = model.generate_content(
                [
                    {
                        "mime_type": "image/png",
                        "data": image_bytes
                    },
                    """ You are an OCR and image-context assistant. Return exactly two sentences and nothing else.
1. First sentence: verbatim extract of all text in the image.
2. Second sentence: a concise explanation of the image’s context.
Do not include headings, labels, bullet points, or extra commentary."""
                ]
            )
            print(response.text.strip())
            time.sleep(5)  # Rate limit spacing
            return response.text.strip()

        except UnidentifiedImageError:
            print(f"[Gemini OCR Error] Skipped invalid image (not recognized format)")
            return ""

        except Exception as e:
            attempt += 1
            if "429" in str(e) or "Rate limit" in str(e):
                wait_time = 90 + random.uniform(0, 15)
                print(f"[Rate Limit] Attempt {attempt}/{max_retries} — Waiting {wait_time:.1f}s before retrying...")
                time.sleep(wait_time)
            else:
                print(f"[Gemini OCR Error] Attempt {attempt}/{max_retries} — Error: {e}")
                time.sleep(5)  # Brief wait for transient errors

    print("[Gemini OCR Error] Max retries exceeded. OCR failed.")
    return ""

# --- SETUP ---
OUTPUT_FILE = "tds_discourse_data.json"
DISCOURSE_URL = "https://discourse.onlinedegree.iitm.ac.in"
COURSE_CATEGORY_SLUG = "courses/tds-kb/34" # Specific slug for "Tools in Data Science"
DISCOURSE_COOKIE_T=os.getenv("DISCOURSE_COOKIE_T")
DISCOURSE_COOKIE_SESSION=os.getenv("DISCOURSE_COOKIE_SESSION")
# --- REVISED: Fetch cookies securely from environment variables ---
browser_cookies = {
    '_t': DISCOURSE_COOKIE_T,
    '_forum_session': DISCOURSE_COOKIE_SESSION
}

session = create_session_with_browser_cookies(DISCOURSE_URL, browser_cookies)

# Verify login by checking the current session
print("Verifying authentication...")
response = session.get(f"{DISCOURSE_URL}/session/current.json", timeout=20)
if response.status_code != 200 or "current_user" not in response.json():
    print("Authentication failed. Please check your cookies in the .env file.")
    exit()
print("Authentication successful.")


# Define time range for posts
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 4, 15)

# Get topics from the specified category
topics = []
page = 0
print(f"Fetching topics from category '{COURSE_CATEGORY_SLUG}'...")
for i in range(15):
    url = f"{DISCOURSE_URL}/c/{COURSE_CATEGORY_SLUG}.json?page={page}"
    try:
        resp = session.get(url, timeout=30)
        resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch topics on page {page}: {e}")
        break

    data = resp.json()
    page_topics = data.get('topic_list', {}).get('topics', [])
    if not page_topics:
        print("No more topics found.")
        break

    topics.extend(page_topics)
    print(f"Fetched {len(page_topics)} topics from page {page}. Total: {len(topics)}")
    page += 1
    time.sleep(1) # Be polite to the server

print(f"\nFetched {len(topics)} total topics.")

# Fetch and process posts within the date range
all_posts = []
print("Collecting posts from topics, processing images to base64, and running OCR...")
BACKUP_INTERVAL = 50
for topic in tqdm(topics, desc="Processing topics"):
    topic_id = topic['id']
    posts = fetch_all_posts_from_topic(topic_id)

    for post in posts:
        try:
          created_at = datetime.strptime(post['created_at'], "%Y-%m-%dT%H:%M:%S.%fZ")
        except (ValueError, TypeError):
            continue # Skip post if date format is wrong or missing

        if start_date <= created_at < end_date:
            post_content_html = post.get("cooked", "")
            soup = BeautifulSoup(post_content_html, 'html.parser')

            images_base64 = []
            extracted_image_text = ""

            for img_tag in soup.find_all('img'):
                img_url = img_tag.get('src')
                if not img_url:
                    continue
                if 'avatar' in img_url or img_url.endswith('.svg'):
                    continue
                if img_url.startswith('/'):
                    img_url = DISCOURSE_URL + img_url

                try:
                    img_response = session.get(img_url, timeout=20)
                    img_response.raise_for_status()
                    image_data = img_response.content

                    # Validate image before converting to base64 or OCR
                    img = Image.open(io.BytesIO(image_data))
                    img.verify()



                    # Skip images that are too small (likely emojis or icons)
                    if img.width < 32 or img.height < 32:
                        tqdm.write(f"  [Skipped] Tiny image ({img.width}x{img.height}) in post {post.get('id')}")
                        continue



                    # Optionally: Skip nearly grayscale or low-info images
                    if img.getbands() == ('L',) and len(set(img.getdata())) < 10:
                        tqdm.write(f"  [Skipped] Low-info grayscale image in post {post.get('id')}")
                        continue

                    img = Image.open(io.BytesIO(image_data))  # Reopen after verify

                    # Encode to base64
                    base64_bytes = base64.b64encode(image_data)
                    base64_string = base64_bytes.decode('utf-8')
                    images_base64.append(base64_string)

                    # OCR
                    # ocr_text = get_text_from_image_data(image_data)
                    if len(image_data) < 2000:
                        tqdm.write(f"[Skip OCR] Too small: {len(image_data)} bytes")
                        continue

                    # if ocr_text:
                    #     extracted_image_text += ocr_text + "\n---\n"

                except UnidentifiedImageError:
                    tqdm.write(f"  [Image Error] Skipped: not a valid image for post {post.get('id')}")
                except Exception as img_e:
                    tqdm.write(f"  [Image Error] for post {post.get('id')}: {img_e}")

            all_posts.append({
                "source": "discourse",
                "id": f"discourse_{post.get('id')}",
                "title": topic.get("title", "Untitled Topic"),
                "content_text": soup.get_text(separator="\n", strip=True),
                "images_base64": images_base64, # <-- SAVING THE LIST OF BASE64 STRINGS
                "extracted_image_text": extracted_image_text.strip(),
                "url": f"{DISCOURSE_URL}/t/{topic_id}/{post.get('post_number', 1)}",
                "metadata": {
                    "username": post.get("username", "unknown"),
                    "created_at": created_at.isoformat(),
                    "topic_id": topic_id,
                    "post_number": post.get("post_number", 0)
                }
            })
            if len(all_posts) % BACKUP_INTERVAL == 0:
              with open("backup.json", "w", encoding="utf-8") as f:
                  json.dump(all_posts, f, indent=2, ensure_ascii=False)
              print(f"[💾] Backup saved with {len(all_posts)} posts")
    time.sleep(1)

print(f"\nCollected {len(all_posts)} posts.")

# Save the unified data to a file
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_posts, f, indent=2, ensure_ascii=False)

print(f"Scraping complete. Saved {len(all_posts)} posts to {OUTPUT_FILE}")

Verifying authentication...
Authentication successful.
Fetching topics from category 'courses/tds-kb/34'...
Fetched 30 topics from page 0. Total: 30
Fetched 30 topics from page 1. Total: 60
Fetched 30 topics from page 2. Total: 90
Fetched 30 topics from page 3. Total: 120
Fetched 30 topics from page 4. Total: 150
Fetched 30 topics from page 5. Total: 180
Fetched 30 topics from page 6. Total: 210
Fetched 30 topics from page 7. Total: 240
Fetched 30 topics from page 8. Total: 270
Fetched 30 topics from page 9. Total: 300
Fetched 30 topics from page 10. Total: 330
Fetched 30 topics from page 11. Total: 360
Fetched 30 topics from page 12. Total: 390
Fetched 30 topics from page 13. Total: 420
Fetched 30 topics from page 14. Total: 450

Fetched 450 total topics.


Processing topics:   4%|▎         | 16/450 [00:55<30:33,  4.22s/it]

[💾] Backup saved with 50 posts
[Skip OCR] Too small: 1344 bytes
[💾] Backup saved with 100 posts


Processing topics:   4%|▎         | 16/450 [00:58<30:33,  4.22s/it]

[Skip OCR] Too small: 895 bytes
  [Skipped] Tiny image (690x12) in post 590891


...

[💾] Backup saved with 3300 posts


Processing topics:  40%|███▉      | 178/450 [09:00<06:01,  1.33s/it]

[💾] Backup saved with 3350 posts


Processing topics: 100%|██████████| 450/450 [15:49<00:00,  2.11s/it]



Collected 3380 posts.
Scraping complete. Saved 3380 posts to tds_discourse_data.json
